In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import keras_tuner as kt

def preprocess_data(file_path, num_genres):
    df = pd.read_csv(file_path)

    topEightGenres = df["genre"].value_counts().index[:num_genres]
    df = df[df["genre"].isin(topEightGenres)]
    
    X = df.drop(columns=['Unnamed: 0', 'genre', 'artist_name', 'track_name', 'track_id', 'popularity', 'year'])
    Y = df['genre']
    
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(Y)

    return X, Y, label_encoder



def build_model(hp):

    #Hyperparameters
    units_1 = hp.Int('units_1', min_value=64, max_value=256, step=64)
    activation_1 = hp.Choice('activation_1', ['relu', 'tanh'])
    units_2 = hp.Int('units_2', min_value=32, max_value=128, step=32)
    activation_2 = hp.Choice('activation_2', ['relu', 'tanh'])
    learning_rate = hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])

    print(f"Testing Hyperparameter Combination")
    print(f"Hidden Layer 1: {units_1} units, Activation: {activation_1}")
    print(f"Hidden Layer 2: {units_2} units, Activation: {activation_2}")
    print(f"Learning Rate: {learning_rate}")

    model = keras.Sequential([
        layers.Dense(units_1, activation=activation_1, input_shape=input_shape),
        layers.Dense(units_2, activation=activation_2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


file_path = '/content/drive/MyDrive/spotify_data.csv'

num_genres = [2,3,4,5,6,7,8,9,10]

best_model_params = []
best_model_accuracies = []

for genre_count in num_genres:

  X, Y, label_encoder = preprocess_data(file_path, genre_count)

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

  num_classes = len(np.unique(Y))
  y_train = to_categorical(y_train, num_classes=num_classes)
  y_test = to_categorical(y_test, num_classes=num_classes)

  input_shape = (X_train.shape[1],)

  tuner = kt.RandomSearch(
      build_model,
      objective='val_accuracy',
      max_trials=10,
      executions_per_trial=1,
      directory='tuner_results',
      project_name='music_genre_tuning'
  )

  tuner.search(X_train, y_train, epochs=20, validation_split=0.2, batch_size=128, verbose=1)

  # Get Best Hyperparameters
  best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

  print(f"Best Hidden Layer 1 Units: {best_hps.get('units_1')}")
  print(f"Best Hidden Layer 2 Units: {best_hps.get('units_2')}")
  print(f"Best Activation 1: {best_hps.get('activation_1')}")
  print(f"Best Activation 2: {best_hps.get('activation_2')}")
  print(f"Best Learning Rate: {best_hps.get('learning_rate')}")

  best_model = tuner.hypermodel.build(best_hps)
  best_model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=128, verbose=1)

  test_loss, test_acc = best_model.evaluate(X_test, y_test)
  print(f"Test Accuracy of Best Model: {test_acc:.4f}")

  best_model_params.append((best_hps.get('units_1'), best_hps.get('units_2'),
                            best_hps.get('activation_1'), best_hps.get('activation_2'), best_hps.get('learning_rate')))

  best_model_accuracies.append(test_acc)



# Plotting

In [ ]:
import matplotlib.pyplot as plt

errors = [1- accuracy for accuracy in best_model_accuracies]

def plot_accuracies(errors, num_genres):
    plt.figure(figsize=(8, 5))
    plt.plot(num_genres, errors, marker='o', linestyle='-', color='b', label='Error')
    
    plt.xlabel('Number of Genres')
    plt.ylabel('Error')
    plt.title('Model Error vs. Number of Genres')
    
    plt.xticks(num_genres)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    
    plt.show()

plot_accuracies(errors, num_genres)

for error in errors:
    print(error*100)